In [ ]:
import torch
from torch.nn import functional as F
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, DistributedSampler
import pytorch_lightning as pl
import numpy as np
from torchvision import datasets, transforms
import os

# Pytorch

In [ ]:
BATCH_SIZE = 512
N_EPOCHS = 5
LEARNING_RATE = 1e-3

Давайте загрузм данные и передадим в dataset пайплайн предобработки

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.FashionMNIST(os.getcwd(), train=True, download=False,
                   transform=transform)
train_loader = DataLoader(dataset1, batch_size=BATCH_SIZE, shuffle=False, num_workers=10)

Создадим модель при помощи pytorch-lightening. У него много возможностей, в том числе легкое логирование всего в tensorboard, уже готовый конструктор для обучения моделей

In [ ]:
def accuracy(true, pred):
    return sum(pred == true) / len(pred)

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class LightModel(pl.LightningModule):
    def __init__(self, learning_rate):
        super(LightModel, self).__init__()
        self.sequential_module = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            Flatten(),

            nn.Linear(1152, 64),
            nn.ReLU(inplace=False),
            nn.Linear(64, 32),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.5),
            nn.Linear(32, 10)
        )
        self.criterion = nn.NLLLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        x = self.sequential_module(x)
        x = F.log_softmax(x, dim=-1)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        pred = torch.squeeze(pred)
        return {'loss': loss, 'accuracy': accuracy(pred, y)}
    
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['accuracy'] for x in outputs]).mean()
        print(f'Train loss = {avg_loss:.4f}, train accuracy = {avg_acc:.4f}')
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

Trainer чем-то похож на обучение моделей в keras. gpus=None говорит о том, что обучать мы будем все на CPU

In [ ]:
trainer_cpu = pl.Trainer(gpus=None, max_epochs=N_EPOCHS, progress_bar_refresh_rate=0)
nn_model = LightModel(LEARNING_RATE)

In [ ]:
%time trainer_cpu.fit(nn_model, train_loader)

In [ ]:
trainer_cpu.model

Предыдущий вариант был долгим...Давайте возьмем гпу с индексом 0.

In [ ]:
trainer_gpu = pl.Trainer(gpus=[0], max_epochs=N_EPOCHS, progress_bar_refresh_rate=0)
nn_model = LightModel(LEARNING_RATE)

In [ ]:
%time trainer_gpu.fit(nn_model, train_loader)

Намного лучше. А что если взять 2 гпу? Для этого нужно немного переписать код, так как мы будем использовать стратегию dp - батч делится на равные части между всеми задействованными gpu

In [ ]:
class LightModel(pl.LightningModule):
    def __init__(self, learning_rate):
        super(LightModel, self).__init__()
        self.sequential_module = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2),

            Flatten(),

            nn.Linear(1152, 64),
            nn.ReLU(inplace=False),
            nn.Linear(64, 32),
            nn.ReLU(inplace=False),
            nn.Dropout(p=0.5),
            nn.Linear(32, 10)
        )
        self.criterion = nn.NLLLoss()
        self.learning_rate = learning_rate

    def forward(self, x):
        x = self.sequential_module(x)
        x = F.log_softmax(x, dim=-1)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        pred = torch.squeeze(pred)
        return {'loss': loss, 'accuracy' : accuracy(pred, y)}
        
    def training_step_end(self, outputs):
        # only use when  on dp
        avg_loss = outputs['loss'].mean()
        avg_accuracy = outputs['accuracy'].mean()
        return {'loss': avg_loss, 'accuracy' : avg_accuracy}
        print(f'Train loss = {avg_loss:.4f}, accuracy = {avg_accuracy:.4f}')
        
    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['accuracy'] for x in outputs]).mean()
        print(f'Train loss = {avg_loss:.4f}, train accuracy = {avg_acc:.4f}')
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

GPU можно выбирать, либо сказать сколько хотим взять и будут выбраны наименее загруженные

In [ ]:
trainer_gpu = pl.Trainer(gpus=2, max_epochs=N_EPOCHS, progress_bar_refresh_rate=0, accelerator='dp', auto_select_gpus=True) #gpus=[0, 1]
nn_model = LightModel(LEARNING_RATE)

In [ ]:
train_loader = DataLoader(dataset1, batch_size=2*BATCH_SIZE, shuffle=False, num_workers=10)

In [ ]:
%time trainer_gpu.fit(nn_model, train_loader)

# Tensorflow

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
tf.__version__

In [ ]:
BATCH_SIZE = 512
N_EPOCHS = 5
LEARNING_RATE = 1e-3

Загрузим также данные fashion_mnist в tensorflow

In [ ]:
path_train = os.path.join(os.getcwd(), 'FashionMNIST/processed/training.pt')
path_test = os.path.join(os.getcwd(), 'FashionMNIST/processed/test.pt')

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
#(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train, y_train = torch.load(path_train)
x_train, y_train = x_train.numpy(), y_train.numpy()
x_test, y_test = torch.load(path_test)
x_test, y_test = x_test.numpy(), y_test.numpy()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Модель возьмем такую же, попроуем воспроизвести)

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(16, kernel_size=(3, 3), padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
        layers.ReLU(),
        layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        layers.Flatten(),
        layers.Dense(64),
        layers.ReLU(),
        layers.Dense(32),
        layers.ReLU(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Параметров вроде столько же, все похоже. Для начала обучим на CPU

In [ ]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
%%time
with tf.device('/CPU:0'):
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)

На GPU первый запуск очень долгий. Во время него идет процесс резрвирования памяти на gpu, перенос данных и весов + как мы уже знаем, первый запуск чего-либо на GPU часто медленный

In [ ]:
%%time
with tf.device('/GPU:0'):
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS)

Обучать на нескольких GPU в рамках однйо машины также просто, нужно создать стратегию и внутри нее определить модель. Также можно выбирать колчество GPU и индексы. Используйте столько, сколько нужно, не жадничайте)

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:1", "/gpu:2"])
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
optimizer = keras.optimizers.Adam(lr=LEARNING_RATE)
with strategy.scope():
  # Everything that creates variables should be under the strategy scope.
  # In general this is only model construction & `compile()`.
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(16, kernel_size=(3, 3), padding="same"),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
            layers.ReLU(),
            layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

            layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
            layers.ReLU(),
            layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

            layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
            layers.ReLU(),
            layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

            layers.Flatten(),
            layers.Dense(64),
            layers.ReLU(),
            layers.Dense(32),
            layers.ReLU(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

Очень долгий первый запуск. Ну логично, создаются полные копии сетки на 2х GPU

In [ ]:
model.fit(x_train, y_train, batch_size=2*BATCH_SIZE, epochs=N_EPOCHS)

# Домашнее задание \ проект

В данном проекте вам нужно будет провести стандартную работу - загрузить данные, поисследовать их, сделать модельку. Сразу оговорюсь, что подбор параметров, интересные архитектуры...на это я внимания обращать не буду, у нас курс не про ML. Главное - это использования GPU. Что нужно?

    1) Загрузить данны при помощи cudf\dask-cudf
    2) Произвести базовый анализ данных (распределение классов, средние значения фичей по классам и вот это вот все). Любые преобразования данных на ваше усмотрение (только комментируйте=) )
    3) Попробуйте в ходе анализа \ обработки найти применение cuda.jit и cupy (достаточно по 1 примеру, даже искуственному)
    4) Взять два метода снижения размерности из cuml (на лекции был UMAP, еще есть tSNE, tSVD, PCA...), обучить их и визуализировать наши классы в пространстве размерности 2 (раскрасить классы будет хорошей идеей)
    5) Обучите нейронку на GPU\Multi-GPU, а также какую-нибудь еще модельку на GPU\Multi-GPU.

Если в пункте 5 будет Multi-GPU - это дополнительные баллы, а также если еще добавите, например, метод кластеризации просто для сравнения =)

P.S. прошу в коде давать комментарии того, что делаете. Пункты 4 и 5 также зачту за домашку 3 занятия.

Данные находятся в файле Train_Set_90621.csv
Amount Defaulted - эту переменную нужно удалить=)

И все на GPU, никаких любимых pandas, numpy и прочего)) Думаю, из описания понятно, как я буду оценивать работу)

Цель всего этого - чтобы вы побольше потыкали GPU